In [1]:
import numpy as np
import time
import sys
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import tensorflow as tf
import torchvision
import torch
import pandas as pd

from sklearn.metrics import accuracy_score

from torch.utils.data import DataLoader
from torch import optim
from torch.autograd import Variable

# IMPORTING FUNCTIONS

In [2]:
from CNN_PGD import *

# IMPORTING DATA

In [3]:
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = ToTensor(),
)

testing_data = datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = ToTensor()
)

In [4]:
BATCH_SIZE = 100

data_loaders = {
    'train' : DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=1),
    'test'  : DataLoader(testing_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=1),
}

# PARAMETERS

In [5]:
# CNN Training Params
EPOCHS = 10

# Testing
epses = [0, 0.1, 0.2, 0.3, 0.45]
TESTING_ITERATIONS = 40
TESTING_ALPHA = 0.01

# BASELINE

In [32]:
# TRAINING

classifier_baseline = CNN()
optimizer_baseline = optim.Adam(classifier_baseline.parameters(), lr = LEARNING_RATE)

(train_loss_baseline, train_accuracy_baseline) = train(cnn_model = classifier_baseline, 
                                                       data_load = data_loaders, 
                                                       optim = optimizer_baseline,
                                                       EPOCHS = EPOCHS)

Epoch [1/10], Progress [200/600], Loss: 0.1859
Epoch [1/10], Progress [400/600], Loss: 0.0506
Epoch [1/10], Progress [600/600], Loss: 0.0506
Epoch [2/10], Progress [200/600], Loss: 0.0520
Epoch [2/10], Progress [400/600], Loss: 0.0513
Epoch [2/10], Progress [600/600], Loss: 0.1541
Epoch [3/10], Progress [200/600], Loss: 0.0477
Epoch [3/10], Progress [400/600], Loss: 0.1275
Epoch [3/10], Progress [600/600], Loss: 0.0807
Epoch [4/10], Progress [200/600], Loss: 0.0164
Epoch [4/10], Progress [400/600], Loss: 0.0673
Epoch [4/10], Progress [600/600], Loss: 0.0038
Epoch [5/10], Progress [200/600], Loss: 0.0039
Epoch [5/10], Progress [400/600], Loss: 0.0428
Epoch [5/10], Progress [600/600], Loss: 0.0275
Epoch [6/10], Progress [200/600], Loss: 0.0006
Epoch [6/10], Progress [400/600], Loss: 0.0297
Epoch [6/10], Progress [600/600], Loss: 0.1255
Epoch [7/10], Progress [200/600], Loss: 0.2227
Epoch [7/10], Progress [400/600], Loss: 0.1285
Epoch [7/10], Progress [600/600], Loss: 0.0388
Epoch [8/10],

In [33]:
(test_loss_baseline, test_accuracy_baseline) = getLossAccuracy(classifier_baseline, data_loaders, 'test', details=True)

test loss: 0.08035
test accuracy: 0.98000


In [34]:
# SAVE DATA TO EXCEL

data = {'Testing': [test_loss_baseline, test_accuracy_baseline]}

for i, (tr_loss, tr_acc) in enumerate(zip(train_loss_baseline, train_accuracy_baseline)):
    data['Training_epoch'+str(i)] = [tr_loss, tr_acc]

df_baseline = pd.DataFrame(data, index=['Loss', 'Accuracy'])
df_baseline = df_baseline.transpose()
df_baseline.to_csv("output\Baseline.csv")

# FIRST CLASSIFIER

In [35]:
# PARAMETERS (for PGD)

eps = 0.3
alpha = 0.02
iterations = 20

In [36]:
classifier_one = CNN()
optimizer_one = optim.Adam(classifier_one.parameters(), lr = LEARNING_RATE)  

In [37]:
# TRAINING

(one_orginal_images, one_pgd_images, one_pgd_labels, 
 one_training_loss, one_training_accuracy, 
 one_adv_training_loss, one_adv_training_accuracy) = pgd_train(cnn_model = classifier_one, 
                                                               data_load = data_loaders, 
                                                               optim = optimizer_one, 
                                                               pgd_function = untargeted_pgd_attack, 
                                                               EPOCHS = EPOCHS, 
                                                               eps = eps, 
                                                               alpha = alpha,
                                                               iterations = iterations)

Epoch [1/10], Progress [200/600], Loss: 2.3059
Epoch [1/10], Progress [400/600], Loss: 2.3033
Epoch [1/10], Progress [600/600], Loss: 2.3003
Epoch [2/10], Progress [200/600], Loss: 2.3077
Epoch [2/10], Progress [400/600], Loss: 2.2990
Epoch [2/10], Progress [600/600], Loss: 2.3039
Epoch [3/10], Progress [200/600], Loss: 2.3122
Epoch [3/10], Progress [400/600], Loss: 2.3039
Epoch [3/10], Progress [600/600], Loss: 2.3047
Epoch [4/10], Progress [200/600], Loss: 2.3067
Epoch [4/10], Progress [400/600], Loss: 2.2954
Epoch [4/10], Progress [600/600], Loss: 2.3128
Epoch [5/10], Progress [200/600], Loss: 2.2980
Epoch [5/10], Progress [400/600], Loss: 2.2948
Epoch [5/10], Progress [600/600], Loss: 2.2862
Epoch [6/10], Progress [200/600], Loss: 2.3018
Epoch [6/10], Progress [400/600], Loss: 2.3091
Epoch [6/10], Progress [600/600], Loss: 2.2967
Epoch [7/10], Progress [200/600], Loss: 2.3057
Epoch [7/10], Progress [400/600], Loss: 2.3031
Epoch [7/10], Progress [600/600], Loss: 2.3016
Epoch [8/10],

In [39]:
# TESTING

one_test_accuracies = []
one_adv_test_accuracies = []

for test_eps in epses:
    
    print("_____________________________________________ EPS:"+str(test_eps))

    (one_test_accuracy, one_adv_test_accuracy) = pgd_test(cnn_model = classifier_one, 
                                                          data_load = data_loaders, 
                                                          pgd_function = untargeted_pgd_attack, 
                                                          eps = test_eps, 
                                                          alpha = TESTING_ALPHA,
                                                          iterations = TESTING_ITERATIONS,
                                                          details=True)
    
    one_test_accuracies.append(one_test_accuracy)
    one_adv_test_accuracies.append(one_adv_test_accuracy)

_____________________________________________ EPS:0
Accuracy on original Test Set: 0.15000
Accuracy of Perturbed Test Set: 0.15000
_____________________________________________ EPS:0.1
Accuracy on original Test Set: 0.12000
Accuracy of Perturbed Test Set: 0.10280
_____________________________________________ EPS:0.2
Accuracy on original Test Set: 0.07000
Accuracy of Perturbed Test Set: 0.10280
_____________________________________________ EPS:0.3
Accuracy on original Test Set: 0.08000
Accuracy of Perturbed Test Set: 0.10280
_____________________________________________ EPS:0.45
Accuracy on original Test Set: 0.06000
Accuracy of Perturbed Test Set: 0.10280


In [40]:
# SAVE DATA TO EXCEL

data = {}

for i, (test, pert) in enumerate(zip(one_test_accuracies, one_adv_test_accuracies)):
    data['Testing_eps'+str(epses[i])] = [test, pert]

for i, (train, pert) in enumerate(zip(one_training_accuracy, one_adv_training_accuracy)):
    data['Training_epoch'+str(i)] = [train, pert]

df_one = pd.DataFrame(data, index=['Original_Data', 'Perturbed_Data'])
df_one = df_one.transpose()
df_one.to_csv("output\Classifier_One.csv")

# SECOND CLASSIFIER

In [9]:
# PARAMETERS (for PGD)

eps = 0.3
alpha = 0.5
iterations = 1

In [10]:
classifier_two = CNN()
optimizer_two = optim.Adam(classifier_two.parameters(), lr = LEARNING_RATE) 

In [11]:
# TRAINING

(two_orginal_images, two_pgd_images, two_pgd_labels, 
 two_training_loss, two_training_accuracy, 
 two_adv_training_loss, two_adv_training_accuracy) = pgd_train(cnn_model = classifier_two, 
                                                               data_load = data_loaders, 
                                                               optim = optimizer_two, 
                                                               pgd_function = untargeted_pgd_attack, 
                                                               EPOCHS = EPOCHS, 
                                                               eps = eps, 
                                                               alpha = alpha,
                                                               iterations = iterations)

Epoch [1/10], Progress [200/600], Loss: 2.3002
Epoch [1/10], Progress [400/600], Loss: 2.2985
Epoch [1/10], Progress [600/600], Loss: 2.3032
Epoch [2/10], Progress [200/600], Loss: 2.2973
Epoch [2/10], Progress [400/600], Loss: 2.2898
Epoch [2/10], Progress [600/600], Loss: 2.2990
Epoch [3/10], Progress [200/600], Loss: 2.3057
Epoch [3/10], Progress [400/600], Loss: 2.3002
Epoch [3/10], Progress [600/600], Loss: 2.3015
Epoch [4/10], Progress [200/600], Loss: 2.2940
Epoch [4/10], Progress [400/600], Loss: 2.3073
Epoch [4/10], Progress [600/600], Loss: 2.2926
Epoch [5/10], Progress [200/600], Loss: 2.3063
Epoch [5/10], Progress [400/600], Loss: 2.3044
Epoch [5/10], Progress [600/600], Loss: 2.2953
Epoch [6/10], Progress [200/600], Loss: 2.2943
Epoch [6/10], Progress [400/600], Loss: 2.3084
Epoch [6/10], Progress [600/600], Loss: 2.3035
Epoch [7/10], Progress [200/600], Loss: 2.3031
Epoch [7/10], Progress [400/600], Loss: 2.3103
Epoch [7/10], Progress [600/600], Loss: 2.3157
Epoch [8/10],

In [12]:
# TESTING

two_test_accuracies = []
two_adv_test_accuracies = []

for test_eps in epses:
    
    print("_____________________________________________ EPS:"+str(test_eps))

    (two_test_accuracy, two_adv_test_accuracy) = pgd_test(cnn_model = classifier_two, 
                                                          data_load = data_loaders, 
                                                          pgd_function = untargeted_pgd_attack, 
                                                          eps = test_eps, 
                                                          alpha = TESTING_ALPHA,
                                                          iterations = TESTING_ITERATIONS,
                                                          details=True)
    
    two_test_accuracies.append(two_test_accuracy)
    two_adv_test_accuracies.append(two_adv_test_accuracy)
    
#print("Testing accuracies:(for growing eps)" + str(two_test_accuracies))
#print("Perturbed Testing accuracies:(for growing eps)" + str(two_adv_test_accuracies))

_____________________________________________ EPS:0
Accuracy on original Test Set: 0.15000
Accuracy of Perturbed Test Set: 0.15000
_____________________________________________ EPS:0.1
Accuracy on original Test Set: 0.13000
Accuracy of Perturbed Test Set: 0.11350
_____________________________________________ EPS:0.2
Accuracy on original Test Set: 0.11000
Accuracy of Perturbed Test Set: 0.11350
_____________________________________________ EPS:0.3
Accuracy on original Test Set: 0.12000
Accuracy of Perturbed Test Set: 0.11350
_____________________________________________ EPS:0.45
Accuracy on original Test Set: 0.08000
Accuracy of Perturbed Test Set: 0.11350


In [21]:
# SAVE DATA TO EXCEL

data = {}

for i, (test, pert) in enumerate(zip(two_test_accuracies, two_adv_test_accuracies)):
    data['Testing_eps'+str(epses[i])] = [test, pert]

for i, (train, pert) in enumerate(zip(two_training_accuracy, two_adv_training_accuracy)):
    data['Training_epoch'+str(i)] = [train, pert]

df_two = pd.DataFrame(data, index=['Original_Data', 'Perturbed_Data'])
df_two = df_two.transpose()
df_two.to_csv("output\Classifier_Two.csv")

In [ ]:
# GET A FEW IMAGES (UNPERTURBED, PERTURBED FROM TRAINING SET)

# THIRD CLASSIFIER

In [6]:
# PARAMETERS (for PGD)

eps = 0.3
alpha = 0.02
iterations = 20

In [7]:
classifier_three = CNN()
optimizer_three = optim.Adam(classifier_three.parameters(), lr = LEARNING_RATE) 

NameError: name 'LEARNING_RATE' is not defined

In [ ]:
# TRAINING

(three_orginal_images, three_pgd_images, three_pgd_labels, 
 three_training_loss, three_training_accuracy, 
 three_adv_training_loss, three_adv_training_accuracy) = pgd_train(cnn_model = classifier_three, 
                                                               data_load = data_loaders, 
                                                               optim = optimizer_three, 
                                                               pgd_function = targeted_pgd_attack, 
                                                               EPOCHS = EPOCHS, 
                                                               eps = eps, 
                                                               alpha = alpha,
                                                               iterations = iterations)

In [ ]:
# TESTING

epses = [0, 0.1, 0.2, 0.3, 0.45]

(three_test_accuracy, three_adv_test_accuracy) = pgd_test(cnn_model = classifier_three, 
                                                      data_load = data_loaders, 
                                                      pgd_function = targeted_pgd_attack, 
                                                      eps = eps, 
                                                      alpha = alpha,
                                                      iterations = iterations,
                                                      details=True)